In [1]:
import pandas as pd
import numpy as np

In [2]:
data1 = {'X48': [5464, 4568, 48952, 4555], 'X52': [np.nan,56782,537534,54354], 'X48_shift1': [np.nan,15,5,5], 'X48_shift2': [534,53,5,456]}
data2 = {'X48': [5464, 4568, 48952, 4555], 'X52': [np.nan,56782,537534,54354]}

In [3]:
df1 = pd.DataFrame(data1, columns = ['X48', 'X52', 'X48_shift1', 'X48_shift2'])
df2 = pd.DataFrame(data2, columns = ['X48', 'X52'])

In [4]:
X_train = df1
Y_train = df2

In [5]:
X_train

,X48,X52,X48_shift1,X48_shift2
0,5464,NaN,NaN,534
1,4568,56782.0,15.0,53
2,48952,537534.0,5.0,5
3,4555,54354.0,5.0,456


In [6]:
Y_train

,X48,X52
0,5464,NaN
1,4568,56782.0
2,48952,537534.0
3,4555,54354.0


In [7]:
for X in df2.columns:
    X_train.drop('{}'.format(X), axis=1, inplace =True)
    Y_train['{}'.format(X)].dropna(axis=1)

ValueError: No axis named 1 for object type <class 'pandas.core.series.Series'>